# Network Visualization

This notebook example will demonstrate how to view **MASSpy** models on network maps using [Escher](https://escher.github.io/#/) visualization tool <cite data-cite="KDE+15">(King et al., 2015)</cite>

Escher must already be installed into the environment.

## Viewing Model Data with Escher

The **MASSpy** package also comes with some maps for testing purposes.

In [ ]:
# Disable gurobi logging output for this notebook.
try:
    import gurobipy
    gurobipy.setParam("OutputFlag", 0)
except ImportError:
    pass

from os.path import join

import numpy as np

import mass.test

maps_dir = mass.test.MAPS_DIR

# To view available maps
print("\n".join(name for name in mass.test.view_test_maps()))

# Load the glycolysis and hemoglobin models, then merge them
glycolysis = mass.test.create_test_model("Glycolysis")
hemoglobin = mass.test.create_test_model("Hemoglobin")
model = glycolysis.merge(hemoglobin, inplace=False)

# Set the path to the map file
map_filepath = join(maps_dir, "glycolysis_hemoglobin_map.json")

The primary object for viewing Escher maps is the `escher.Builder`, a Jupyter widget that can be viewed in a Jupyter notebook. 

In [ ]:
from escher import Builder

To load an existing map, the path to the JSON file of the Escher map is passed to the `map_json` argument of the  `Builder`. The `MassModel` can be loaded using the `model` argument.

In [ ]:
escher_builder = Builder(
    model=model,
    map_json=map_filepath)
escher_builder

### Reaction Data

Reaction data can be displayed on the Escher map by using a dictionary containing reaction identifiers and the values to be mapped to  corresponding reaction arrows and labels. The `dict` can be passed to the `reaction_data` argument upon initialization of the builder.

For example, to display the steady state fluxes on the map:

In [ ]:
initial_flux_data = {
    reaction.id: flux
    for reaction, flux in model.steady_state_fluxes.items()}

# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=model,
    map_json=map_filepath,
    reaction_data=initial_flux_data)

# Display map in notebook
escher_builder

The color and size of the data scale can be altered by passing a tuple of at least two dictionaries. Each dictionary is considered a "stop" that defines the color and size at or near that particular value in the data set. The `type` key defines the type for the stop, the `color` key defines the color of the arrow, and `size` defines the thickness of the arrow.

In [ ]:
# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=model,
    map_json=map_filepath,
    reaction_data=initial_flux_data,
    reaction_scale=(
        {"type": 'min', "color": 'green', "size": 5 },
        {"type": 'value', "value": 1.12, "color": 'purple', "size": 10},
        {"type": 'max', "color": 'blue', "size": 15 }),
)

escher_builder

### Metabolite Data

Metabolite data can also be displayed on the Escher map by using a dictionary containing metabolite identifiers and the values to be mapped to corresponding metabolite nodes and labels. In addition to setting the attributes to apply for the map upon initializing the builder, they can also be set for a map after initialization

For example, to display the metabolite concentrations on the map:

In [ ]:
initial_conc_data = {
    metabolite.id: round(conc, 8)
    for metabolite, conc in model.initial_conditions.items()}

# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=model,
    map_json=map_filepath, 
    metabolite_data=initial_conc_data)

escher_builder

The secondary metabolites can be removed by setting `hide_secondary_metabolites` as `True` to provide a cleaner visualization of the primary metabolites in the network.

In [ ]:
escher_builder.hide_secondary_metabolites = True

Note that made will affect the already displayed map. Here, a preset scale will be applied to the metabolite concentrations.

In [ ]:
escher_builder.metabolite_scale_preset = "RdYlBu"

## Animating Simulation Results with Escher

Simulation results can also be displayed on an escher map. Using the reactive options of Escher, the results of a simulation can be animated. Here is an example of the simulation results with an oscillating oxygen uptake rate.

In [ ]:
from time import sleep

from mass import MassConfiguration, Simulation

The simulation is performed and results are obtained:

In [ ]:
mc = MassConfiguration()
mc.decimal_precision=8

In [ ]:
sim = Simulation(model, verbose=True, variable_step_size=True)

tstart = 0
tfinal = 20
numpoints = 1000

conc_sol, flux_sol = sim.simulate(
    model, time=(tstart, tfinal, numpoints),
    perturbations={"o2_b": "(70 + 30*sin(120*pi*t))*2.8684*1e-4"},
    interpolate=False, decimal_precision=True)

The Escher map is loaded and displayed:

In [ ]:
escher_builder = Builder(
    model=model,
    map_json=map_filepath,
    reaction_scale=(
        {"type": "min", "color": "rgb(255, 0, 0)", "size": 12},
        {"type": "value", "value": 0, "color": "rgb(170, 0, 85)", "size": 16},
        {"type": "value", "value": 2.24, "color": "rgb(127, 0, 170)", "size": 20},
        {"type": "max", "color": "rgb(0, 0, 255)", "size": 24}
    ),
    hide_secondary_metabolites=True)
escher_builder

The simulation results are looped through and as the loop runs, the Escher map will change to reflect the state of the simulation at that particular point in time.

In [ ]:
for i in range(int(numpoints)):
    escher_builder.metabolite_data = {
        met: conc[i] for met, conc in conc_sol.items()}
    escher_builder.reaction_data = {
        rxn: flux[i] for rxn, flux in flux_sol.items()}
    sleep(0.05)